In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime
import numpy as np

data = pd.read_table('Ma_ten_truong_DH.txt')
matruong=[]
kyhieumatruong=[]
with open('Ma_ten_truong_DH.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
for i in lines:
    matruong.append(i.split(' - ')[0]) #0 lấy trước dấu '-' 1 sẽ lấy sau '-'
for i in matruong:
    kyhieumatruong.append(i)

In [2]:
# Đường dẫn tới ChromeDriver
service = Service(r"C:\Users\QUANGHONEY\Desktop\Code\NCKH_Crawl_Data\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Mở trang web
driver.get('https://diemthi.vnexpress.net/tra-cuu-dai-hoc')

# Tìm ô input và nhập mã trường
#input_field = driver.find_element(By.CSS_SELECTOR, "#input_college")
input_field = driver.find_element(By.CSS_SELECTOR, 'input[type="text"]')

input_field.clear()
input_field.send_keys(f'{kyhieumatruong[0]}')
input_field.send_keys(Keys.RETURN)

time.sleep(5)# nghỉ 60s để load nốt trang nếu không sẽ lấy data của trang chưa load lại hết chỉ lấy đươcj 59

Link = driver.find_elements(By.XPATH, "//div[@class='lookup__result-name']//a")
print(len(Link))
first_link=Link[0]
#Chuyển đến tab khác để tránh kích hoạt quảng cáo
driver.execute_script("window.open(arguments[0].href, '_blank');", first_link)
# Chuyển sự theo dõi của selenium sang tab mới nếu không chuyển selenium sẽ vẫn ở tab cũ tab gõ dcn window_handles[-1] giúp chuyển sang tab cuối cùng theo định dạng id các tab
driver.switch_to.window(driver.window_handles[-1])
time.sleep(30)# nghỉ 60s để load nốt trang nếu không sẽ lấy data của trang chưa load lại hết chỉ lấy đươcj 59




NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [3]:

# # Tìm dropdown chọn năm
# dropdown = driver.find_element(By.ID, 'select2-select_year-container')  # Sử dụng ID hoặc cách khác để tìm element

# # Nếu dropdown được xây dựng bằng <select>:
# select = Select(dropdown)  # Tạo đối tượng Select

# # Chọn năm bằng cách chọn theo visible text (ví dụ: "Năm 2024")
# select.select_by_visible_text("Năm 2024")

# # Hoặc chọn theo giá trị (nếu có)
# # select.select_by_value('2024')

# # Nếu bạn cần lấy dữ liệu sau khi chọn năm, bạn có thể tiếp tục tương tác với trang web


In [4]:
# # Click vào dropdown chọn năm (ở đây là <span>)
# dropdown = driver.find_element(By.ID, 'select2-select_year-container')
# dropdown.click()

# # Tìm và chọn năm 2024 trong danh sách (tìm theo text hiển thị)
# year_option = driver.find_element(By.XPATH, "//li[text()='2023']")
# year_option.click()

# # Chờ thêm để trang tải dữ liệu sau khi chọn năm (tùy thuộc vào trang, có thể cần chờ vài giây)
# time.sleep(2)


In [5]:
# dropdown.click(): Mở danh sách các năm bằng cách click vào phần tử <span>.
# WebDriverWait: Chờ cho đến khi phần tử tương ứng (năm 2023) có thể click được.
# year_option.click(): Chọn năm 2023 từ danh sách
# # Đợi trang tải và phần tử có thể click
dropdown = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'select2-select_year-container'))
)

# Click vào dropdown để mở danh sách
dropdown.click()
a=2024
# Chờ cho đến khi các mục dropdown hiển thị và tìm mục có text "2023"
year_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f"//li[contains(text(), '{a}')]"))
)

# Click vào mục năm 2023
year_option.click()

# Thêm thời gian chờ nếu cần thiết để trang tải dữ liệu sau khi chọn năm
time.sleep(2)


In [6]:
# Lấy tất cả các rows trong bảng, bỏ qua những hàng có class 'university__benchmark university__benchmark--chart'
div = driver.find_element(By.XPATH, "//table[@class='university__table']")
rows = div.find_elements(By.XPATH, ".//tr[not(contains(@class, 'university__benchmark--chart'))]")
thead = div.find_element(By.XPATH, ".//thead")
ths = thead.find_elements(By.XPATH, ".//tr//th")

#rows = driver.find_elements(By.XPATH, "//table[@class='university__table']//tr[not(contains(@class, 'university__benchmark--chart'))]")
diem=[]
th_data = [th.text for th in ths]

# Duyệt qua các hàng để in dữ liệu
for row in rows:
    # cols = row.find_elements(By.XPATH, ".//td")
    cols = row.find_elements(By.TAG_NAME, 'td')
    row_data = [col.text for col in cols]#Lambda để lưu dữ liệu crawl được theo từng row
    # Thêm dữ liệu vào danh sách theo theo từng row của data
    diem.append(row_data)
    # for col in cols:
    #     diem.append(col.text)
    #     diem.append(',')
    # diem.append('\n')
#Dùng THS làm tiêu đề cho dataframe
df = pd.DataFrame(diem, columns=th_data)
df.drop(0, inplace=True)
df.to_csv(f'diem_{kyhieumatruong[0]}_nam_{a}_vnexpress.csv',index=False)